In [1]:
from platform import python_version
print(python_version())
import pandas as pd
import json
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import geocoder
from geopy.geocoders import Nominatim
import os
from dotenv import load_dotenv # add this line
load_dotenv()
ACS_API_KEY = os.getenv('ACS_API_KEY')
BLS_API_KEY = os.getenv("BLS_API_KEY")

3.9.7


In [27]:
# Name - Investment Byte-sized: Crunching Data for Millennials' Money Moves" 
# sub: Sharing My Journey to Financial Success, One Data Point at a Time
# Sources
# https://pygis.io/docs/d_access_census.html -> Python geocoding
# https://data.gov/ -> A lot of data, hard to find
# https://eintaxid.com/ -> scrape employer
# https://www.nctcog.org/home -> Only for north central texas
# https://www.bls.gov/ -> Nothing interesting
# https://www.hud.gov -> Not relevant
# Chamber of Commerce -> Nothing interesting
# https://censusreporter.org/ -> Nothing interesting
# https://www.citypopulation.de/
# https://www.propertyshark.com/mason/Property/16270076/1371-W-37-Dr-Los-Angeles-CA-90007/ -> $50, 175 property report
# https://www.countyoffice.org/property-records-search/?q=1371-w-37th-dr-los-angeles-ca-90007-ad1 -> Property info
# https://maps.assessor.lacounty.gov/m/ -> For LA only. Assessor data, "property assessor information system", Look into it more
# https://stateline.org/ -> State law related news
# https://www.youtube.com/watch?v=cEsC5hNfPU4
# https://www.youtube.com/watch?v=0Flsg_mzG-M
# https://youtu.be/V0zAvlmzDFc
# https://www.lung.org/research/sota/city-rankings/states/california -> air polution
# https://www.bea.gov/ -> Nothing interesting

# https://www.cato.org/ - nothing interesting
# https://www.fhfa.gov/DataTools/Downloads -> A lot of interesting data, needs more research
# https://goodjobsfirst.org/about/ -> Government subsidy data
# https://www.nasbo.org/home -> Interesting but on a state level
# https://www.ncsl.org/technology-and-communication/state-9-1-1-bill-tracking-database -> nothing interesting
# https://nvca.org/research/regional-data/ -> only this URL. Venture Capital per state
# https://taxfoundation.org/country/united-states/ -> Nothing interesting

# The American Housing Survey (AHS)
# The American Time Use Survey (ATUS)
# Census of Governments
# Census of Jails
# Census of State and Federal Adult Correctional Facilities
# County Business Patterns (CBP) (and ZIP Code Business Patterns)
# Longitudinal Employer-Household Dynamics (LEHD)
# National Crime Victimization Survey
# National Household Education Surveys (NHES)
# Survey of Income and Program Participation (SIPP)
# Quarterly Workforce Indicators (QWI)

# ATTOM Data Solutions, or CoStar Group
# https://www.car.org/en/marketdata/data/housingdata
# https://www.sba.gov/business-guide/plan-your-business/market-research-competitive-analysis#id-free-small-business-data-and-trends
# https://github.com/DenisCarriere/geocoder
# https://www.urban.org/ - Interesting articles

# Cap Rate: Cap Rate = NOI / Market value = 1/multiple: 1-20%
# Net Operating Income (NOI)
# Rent Control - https://www.urban.org/sites/default/files/publication/99646/rent_control._what_does_the_research_tell_us_about_the_effectiveness_of_local_action_1.pdf
# Rent related laws - https://www.nolo.com/legal-encyclopedia/renting-leases-deposits
# https://fred.stlouisfed.org/ - City level data, a lot of variety


In [3]:
# Things to consider: 
# 1. Need to manage property from a drivable distance, housing price appreciation(high demand)-County level
# 2. How many companies are in the county? (what attracts populations)
# 3. How many companies will be in the county?
# 4. How many houses are being built in the county?
# 5. How many vacancies are there in the county?
# 6. Does the County has enough population?
# 7. How does housing price has been in the county?
# 8. what is the county's population density
# ----------------------------------------------------------------------------------------------
# 1. Which city is young?
# 2. What is the city's population density
# 3. City's ethnic diversity
# 4. City's income level
# 5. City's employment
# 6. Price per square foot
# 7. Price appreciation -> long term / short term  on neighboring cities
# For example, Southern part has gone through great appreciation in the long term, but in short term, north is
# experiencing greater house appreication. 
# 8. Time on market - liquidity
# Median Rental Rate - The median rental rate map represents the median price per square foot rental rate within a geographical area.

# ----------------------------------------------------------------------------------------------
# Zip code
# ----------------------------------------------------------------------------------------------
# census tract
# 1. Is there wholefood nearby? within 5 mile 
# 2. Is there starbucks nearby? wihin 1 mile
# 3. Nearest school
# 4. Crime
# 5. Price to rent ratio
# Age of Stock
# ----------------------------------------------------------------------------------------------

# Population growth (preferred: at least or equal to 20% since 2000)
# Income growth (preferred: at least or equal to 30% since 2000)
# Median house/condo ownership (preferred: at least or equal to 40% since 2000)
# Median home price
# Crime table (preferred: under 500)
# Poverty level (preferred: under 20%)
# Unemployment rate (neighborhood not more than 2% than the city)
# Employer diversity (Number of major employer industries)
# Racial diversity (preferred: 2 or more races with at least 25%)
# Major LT institutions (preferred: i.e. military bases, higher education institutions, major medical facilities)
# Regional infrastructure (i.e. airports, train stations, ports, etc.)
# New home builds
# Construction permits (in units or $, housing and multifamily)
# Construction cost ($/sqft)
# Crime
# Schools
# Housing Starts and Building Permits
# - Proximity to local businesses - Whole food, Starbucks
# Vacancy Rates
# Transportion
# Employer
# Current Price Per Square Foot
# Time on Market
# Transaction Count
# Median Rental Rate - The median rental rate map represents the median price per square foot rental rate within a geographical area. 
# Path of Progress - The path of progress map represents all issued new construction single family residential permits, providing an eagle-eye view of where progress is flowing.



# Data Collection

## Variable mapping

In [5]:
# # Data profile variable mapper creating script
# variable_info = {
#     "DP05": range(1,90),
#     "DP04": range(1,144),
#     "DP03": range(1,138),
#     "DP02": range(1,155)
# }
# variable_mapper = {}
# for key, val in variable_info.items():
#     for num in val:
#         res = requests.get(f'https://api.census.gov/data/2021/acs/acs1/profile/variables/{key}_{str(num).zfill(4)}E.json')
#         res = res.json()
#         variable_mapper[res["name"]] = res["label"]
# with open("./data/acs_1_dp_2021_column_mapper.json", "w") as outfile:
#     json.dump(variable_mapper, outfile)

In [6]:
# Comparison Data profile variable mapper creating script
# variable_info = {
#     "CP05": range(1,90),
#     "CP04": range(1,144),
#     "CP03": range(1,138),
#     "CP02": range(1,155)
# }
# variable_mapper = {}
# for key, val in variable_info.items():
#     for num in val:
#         for year in [2017,2018,2019,2021]:
#             res = requests.get(f'https://api.census.gov/data/2021/acs/acs1/cprofile/variables/{key}_{year}_{str(num).zfill(3)}E.json')
#             res = res.json()
#             variable_mapper[res["name"]] = res["label"]
# with open("./data/acs_1_cp_2021_column_mapper.json", "w") as outfile:
#     json.dump(variable_mapper, outfile)

In [7]:
# Comparison Data profile variable mapper creating script
# variable_info = {
#     "CP05": range(1,85),
#     "CP04": range(1,144),
#     "CP03": range(1,138),
#     "CP02": range(1,153)
# }
# variable_mapper = {}
# for key, val in variable_info.items():
#     for num in val:
#         for year in [2013,2014,2015,2016]:
#             res = requests.get(f'https://api.census.gov/data/2016/acs/acs1/cprofile/variables/{key}_{year}_{str(num).zfill(3)}E.json')
#             try:
#                 res = res.json()
#             except:
#                 print(f"{key}_{year}_{str(num).zfill(3)}E")
#                 print(res)
#             variable_mapper[res["name"]] = res["label"]
# with open("./data/acs_1_cp_2016_column_mapper.json", "w") as outfile:
#     json.dump(variable_mapper, outfile)

In [ ]:
with open('./data/acs_1_dp_2021_column_mapper.json') as json_file:
    variable_mapper = json.load(json_file)
with open('./data/acs_1_cp_2021_column_mapper.json') as json_file:
    temp_mapper = json.load(json_file)
variable_mapper.update(temp_mapper)
with open('./data/acs_1_cp_2016_column_mapper.json') as json_file:
    temp_mapper = json.load(json_file)
variable_mapper.update(temp_mapper)
for key, val in variable_mapper.items():
    variable_mapper[key] = "_" + val.replace("!!", "__")\
    .replace(" ", "_")\
    .replace("-", "_")\
    .replace("Estimate__", "")\
    .replace("years", "y")\
    .replace("_or_African_American","")\
    .replace("(", "")\
    .replace(")","")\
    .replace("HOUSING_OCCUPANCY__", "")\
    .replace("YEAR_STRUCTURE_BUILT__", "")\
    .replace("UNITS_IN_STRUCTURE__", "")\
    .replace("HOUSING_TENURE__", "")\
    .replace("YEAR_HOUSEHOLDER_MOVED_INTO_UNIT__", "")\
    .replace("VEHICLES_AVAILABLE__", "")\
    .replace("MORTGAGE_STATUS__", "")\
    .replace("SELECTED_MONTHLY_OWNER_COSTS_", "")\
    .replace("AS_A_PERCENTAGE_OF_HOUSEHOLD_INCOME_", "")\
    .replace("GROSS_RENT_AS_A_PERCENTAGE_OF_HOUSEHOLD_INCOME_", "")\
    .replace("_excluding_units_where_SMOCAPI_cannot_be_computed", "")\
    .replace("_excluding_units_where_GRAPI_cannot_be_computed", "")\
    .replace("GROSS_RENT_GRAPI__", "")\
    .replace("EMPLOYMENT_STATUS__", "")\
    .replace("and_over", "up")\
    .replace("labor_force", "work")\
    .replace("OCCUPATION__", "")\
    .replace("INDUSTRY__", "")\
    .replace(",", "")\
    .replace(".0", "")\
    .replace(".", "")
variable_mapper

## ACS Data collection

In [9]:
# source: ACS
# range: 1 year
# year: 2021
# characteristics: Demographics, social, economic, housing
# type: data profile
# unit: city
# interesting_columns = {
#     "DP05": [1, 5, 6, 7, 8, 9, 10,11,12,13,14,15,16,17,18,33,36,37,38,44,57, 70],
#     "DP04": [num for num in range(1,144) if num not in range(62,80)],
#     "DP03": [num for num in range(1, 138) if num not in range(8,14)],
#     "DP02": [num for num in range(1,105) if num not in range(69,79)]
# }
# combined_df = None
# for key, val in interesting_columns.items():
#     for num in val:
#         variable = f"{key}_{str(num).zfill(4)}E"
#         res = requests.get(f'https://api.census.gov/data/2021/acs/acs1/profile?get=NAME,{variable}&for=place:*&in=state:*&key={ACS_API_KEY}')
#         json_list = res.json()
#         df = pd.DataFrame(json_list[1:], columns = ["Name", variable_mapper[variable], "state", "city"])
#         df[variable_mapper[variable]] = df[variable_mapper[variable]].astype(float)
#         df = df.replace(-999999999, 0)
#         if combined_df is None:
#             combined_df = df.copy()
#             continue
#         df = df.drop(columns="Name")
#         combined_df = combined_df.merge(df, how='left', on=["state", "city"])
#         combined_df.to_csv(f"./data/acs_1_{key}_city_2021.csv", index=False)
#         combined_df = None

# combined_df.to_excel("./data/acs_1_dp_city_2021.xlsx", index=False)

In [56]:
# source: ACS
# range: 1 year
# year: 2021
# characteristics: Demographics, social, economic, housing
# type: Comparison profile
# unit: county
# interesting_columns = {
#     "CP05": [1, 5, 6, 7, 8, 9, 10,11,12,13,14,15,16,17,18,33,36,37,38,44,57, 70],
#     "CP04": [num for num in range(1,144) if num not in range(62,80)],
#     "CP03": [num for num in range(1, 138) if num not in range(8,14)],
#     "CP02": [num for num in range(1,105) if num not in range(69,79)]
# }
# combined_df = None
# for key, val in interesting_columns.items():
#     for num in val:
#         for year in [2017,2018,2019,2021]:
#             variable = f"{key}_{year}_{str(num).zfill(3)}E"
#             res = requests.get(f'https://api.census.gov/data/2021/acs/acs1/cprofile?get=NAME,{variable}&for=county:*&in=state:*&key={ACS_API_KEY}')
#             json_list = res.json()
#             if len(variable_mapper[variable]) > 62:
#                 column_name = variable
#             else:
#                 column_name = variable_mapper[variable]
#             df = pd.DataFrame(json_list[1:], columns = ["Name", column_name, "state", "county"])
#             df[column_name] = df[column_name].astype(float)
#             df = df.replace(-888888888, 0)
#             if combined_df is None:
#                 combined_df = df.copy()
#                 continue
#             df = df.drop(columns="Name")
#             combined_df = combined_df.merge(df, how='left', on=["state", "county"])
#     combined_df.to_csv(f"./data/acs_1_{key}_county_2021.csv", index=False)
#     combined_df = None

In [57]:
# source: ACS
# range: 1 year
# year: 2016
# characteristics: Demographics, social, economic, housing
# type: Comparison profile
# unit: county
# interesting_columns = {
#     "CP05": [1,4,5,6,7,8,9,10,11,12,13,14,15,16,17,28,32,33,39,52,65,81],
#     "CP04": [num for num in range(1,144) if num not in range(62,80)],
#     "CP03": [num for num in range(1, 138) if num not in range(8,14)],
#     "CP02": [num for num in range(1,103) if num not in range(68,78)]
# }
# combined_df = None
# for key, val in interesting_columns.items():
#     for num in val:
#         for year in [2013,2014,2015,2016]:
#             variable = f"{key}_{year}_{str(num).zfill(3)}E"
#             res = requests.get(f'https://api.census.gov/data/2016/acs/acs1/cprofile?get=NAME,{variable}&for=county:*&in=state:*&key={ACS_API_KEY}')
#             json_list = res.json()
#             if len(variable_mapper[variable]) > 62:
#                 column_name = variable
#             else:
#                 column_name = variable_mapper[variable]
#             df = pd.DataFrame(json_list[1:], columns = ["Name", column_name, "state", "county"])
#             df[column_name] = df[column_name].astype(float)
#             df = df.replace(-888888888, 0)
#             if combined_df is None:
#                 combined_df = df.copy()
#                 continue
#             df = df.drop(columns="Name")
#             combined_df = combined_df.merge(df, how='left', on=["state", "county"])
#     combined_df.to_csv(f"./data/acs_1_{key}_county_2016.csv", index=False)
#     combined_df = None

In [54]:
# source: ACS
# range: 1 year
# year: 2021
# characteristics: Demographics, social, economic, housing
# type: Comparison profile
# unit: city
# interesting_columns = {
#     "CP05": [1, 5, 6, 7, 8, 9, 10,11,12,13,14,15,16,17,18,33,36,37,38,44,57, 70],
#     "CP04": [num for num in range(1,144) if num not in range(62,80)],
#     "CP03": [num for num in range(1, 138) if num not in range(8,14)],
#     "CP02": [num for num in range(1,105) if num not in range(69,79)]
# }
# combined_df = None
# for key, val in interesting_columns.items():
#     for num in val:
#         for year in [2017,2018,2019,2021]:
#             variable = f"{key}_{year}_{str(num).zfill(3)}E"
#             res = requests.get(f'https://api.census.gov/data/2021/acs/acs1/cprofile?get=NAME,{variable}&for=place:*&in=state:*&key={ACS_API_KEY}')
#             json_list = res.json()
#             if len(variable_mapper[variable]) > 62:
#                 column_name = variable
#             else:
#                 column_name = variable_mapper[variable]
#             df = pd.DataFrame(json_list[1:], columns = ["Name", column_name, "state", "city"])
#             df[column_name] = df[column_name].astype(float)
#             df = df.replace(-888888888, 0)
#             if combined_df is None:
#                 combined_df = df.copy()
#                 continue
#             df = df.drop(columns="Name")
#             combined_df = combined_df.merge(df, how='left', on=["state", "city"])
#     combined_df.to_csv(f"./data/acs_1_{key}_city_2021.csv", index=False)
#     combined_df = None

In [55]:
# source: ACS
# range: 1 year
# year: 2016
# characteristics: Demographics, social, economic, housing
# type: Comparison profile
# unit: city
# interesting_columns = {
#     "CP05": [1,4,5,6,7,8,9,10,11,12,13,14,15,16,17,28,32,33,39,52,65,81],
#     "CP04": [num for num in range(1,144) if num not in range(62,80)],
#     "CP03": [num for num in range(1, 138) if num not in range(8,14)],
#     "CP02": [num for num in range(1,103) if num not in range(68,78)]
# }
# combined_df = None
# for key, val in interesting_columns.items():
#     for num in val:
#         for year in [2013,2014,2015,2016]:
#             variable = f"{key}_{year}_{str(num).zfill(3)}E"
#             res = requests.get(f'https://api.census.gov/data/2016/acs/acs1/cprofile?get=NAME,{variable}&for=place:*&in=state:*&key={ACS_API_KEY}')
#             json_list = res.json()
#             if len(variable_mapper[variable]) > 62:
#                 column_name = variable
#             else:
#                 column_name = variable_mapper[variable]
#             df = pd.DataFrame(json_list[1:], columns = ["Name", column_name, "state", "city"])
#             df[column_name] = df[column_name].astype(float)
#             df = df.replace(-888888888, 0)
#             if combined_df is None:
#                 combined_df = df.copy()
#                 continue
#             df = df.drop(columns="Name")
#             combined_df = combined_df.merge(df, how='left', on=["state", "city"])
#     combined_df.to_csv(f"./data/acs_1_{key}_city_2016.csv", index=False)
#     combined_df = None

In [ ]:
# source: ACS
# range: 1 year
# year: 2021
# characteristics: Demographics, social, economic, housing
# type: Comparison profile
# unit: Metro politan area
# interesting_columns = {
#     "CP05": [1, 5, 6, 7, 8, 9, 10,11,12,13,14,15,16,17,18,33,36,37,38,44,57, 70],
#     "CP04": [num for num in range(1,144) if num not in range(62,80)],
#     "CP03": [num for num in range(1, 138) if num not in range(8,14)],
#     "CP02": [num for num in range(1,105) if num not in range(69,79)]
# }
# combined_df = None
# for key, val in interesting_columns.items():
#     for num in val:
#         for year in [2017,2018,2019,2021]:
#             variable = f"{key}_{year}_{str(num).zfill(3)}E"
#             res = requests.get(f'https://api.census.gov/data/2021/acs/acs1/cprofile?get=NAME,{variable}&for=metropolitan%20statistical%20area/micropolitan%20statistical%20area:*&key={ACS_API_KEY}')
#             json_list = res.json()
#             if len(variable_mapper[variable]) > 62:
#                 column_name = variable
#             else:
#                 column_name = variable_mapper[variable]
#             df = pd.DataFrame(json_list[1:], columns = ["Name", column_name, "metro_politan_area"])
#             df[column_name] = df[column_name].astype(float)
#             df = df.replace(-888888888, 0)
#             if combined_df is None:
#                 combined_df = df.copy()
#                 continue
#             df = df.drop(columns="Name")
#             combined_df = combined_df.merge(df, how='left', on=["metro_politan_area"])
#     combined_df.to_csv(f"./data/acs_1_{key}_metro_politan_2021.csv", index=False)
#     combined_df = None

In [13]:
# source: ACS
# range: 1 year
# year: 2016
# characteristics: Demographics, social, economic, housing
# type: Comparison profile
# unit: Metro politan area
# interesting_columns = {
#     "CP05": [1,4,5,6,7,8,9,10,11,12,13,14,15,16,17,28,32,33,39,52,65,81],
#     "CP04": [num for num in range(1,144) if num not in range(62,80)],
#     "CP03": [num for num in range(1, 138) if num not in range(8,14)],
#     "CP02": [num for num in range(1,103) if num not in range(68,78)]
# }
# combined_df = None
# for key, val in interesting_columns.items():
#     for num in val:
#         for year in [2013,2014,2015,2016]:
#             variable = f"{key}_{year}_{str(num).zfill(3)}E"
#             res = requests.get(f'https://api.census.gov/data/2016/acs/acs1/cprofile?get=NAME,{variable}&for=metropolitan%20statistical%20area/micropolitan%20statistical%20area:*&key={ACS_API_KEY}')
#             json_list = res.json()
#             if len(variable_mapper[variable]) > 62:
#                 column_name = variable
#             else:
#                 column_name = variable_mapper[variable]
#             df = pd.DataFrame(json_list[1:], columns = ["Name", column_name, "metro_politan_area"])
#             df[column_name] = df[column_name].astype(float)
#             df = df.replace(-888888888, 0)
#             if combined_df is None:
#                 combined_df = df.copy()
#                 continue
#             df = df.drop(columns="Name")
#             combined_df = combined_df.merge(df, how='left', on=["metro_politan_area"])
#     combined_df.to_csv(f"./data/acs_1_{key}_metro_politan_2016.csv", index=False)
#     combined_df = None

## MSA - City mapping

In [147]:
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}
    
# invert the dictionary
abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))

In [217]:
# geolocator = Nominatim(user_agent="asdfasdf")
df_1 = pd.read_csv("./data/acs_1_CP02_city_2016.csv")
df_1 = df_1[["Name", "state", "city"]]
df_2 = pd.read_csv("./data/acs_1_CP02_city_2021.csv")
df_2 = df_2[["Name", "state", "city"]]
df_3 = pd.concat([df_1, df_2])
df_3.drop_duplicates()
# df_3 = df_3.drop_duplicates()
# df_3

# df_7 = pd.read_csv("./data/acs_1_CP02_metro_politan_2021.csv")
# df_7 = df_7["Name"]

# def find_string_with_condition(lst, condition):
#     for string in lst:
#         if condition(string):
#             return string
#     return None

# for idx, row in df_3.iterrows():
#     try:
#         location = geolocator.geocode(row["Name"])
#         address_list = location.address.split(",")
#         result = find_string_with_condition(address_list, lambda s: "County" in s)
#         df_3.loc[idx, "County"] = result.strip().replace(" County", "")
#     except:
#         df_3.loc[idx, "County"] = "NA"

                              Name  state   city
0    Bolingbrook village, Illinois     17   7133
1            Joliet city, Illinois     17  38570
2            Peoria city, Illinois     17  59000
3    Highlands Ranch CDP, Colorado      8  36410
4           Decatur city, Illinois     17  18823
..                             ...    ...    ...
589       Champaign city, Illinois     17  12385
590         Boise City city, Idaho     16   8830
591           Meridian city, Idaho     16  52120
592          Atlanta city, Georgia     13   4000
593    Minneapolis city, Minnesota     27  43000

[594 rows x 3 columns]
                            Name  state   city
0        O'Fallon city, Missouri     29  54074
1       St. Louis city, Missouri     29  65000
2       Passaic city, New Jersey     34  56550
3     Nashua city, New Hampshire     33  50260
4      Rochester city, Minnesota     27  54880
..                           ...    ...    ...
615            Ankeny city, Iowa     19   2305
616         

,Name,state,city
0,"Bolingbrook village, Illinois",17,7133
1,"Joliet city, Illinois",17,38570
2,"Peoria city, Illinois",17,59000
3,"Highlands Ranch CDP, Colorado",8,36410
4,"Decatur city, Illinois",17,18823
...,...,...,...
560,"Conway city, Arkansas",5,15190
567,"South Fulton city, Georgia",13,72122
587,"Port Charlotte CDP, Florida",12,58350
615,"Ankeny city, Iowa",19,2305


In [214]:
city_msa_mapper = {}

df_8 = pd.DataFrame(df_7.str.split(',', expand=True))
df_8['County'] = df_7
df_8 = df_8.rename(columns={0: 'County_name', 1: 'State'})
df_8['State'] = df_8['State'].apply(lambda x: x.split()[0])

for idx, row in df_3.iterrows():
    city_list = row['Name'].split(", ")
    city_name = " ".join(city_list[0].split()[:-1])
    state = city_list[-1]
    
    filtered_df = df_8.loc[(df_8['State'].str.contains(us_state_to_abbrev[state])) & (df_8['County_name'].str.contains(city_name))]
    
    if len(filtered_df) > 0:
        city_msa_mapper[row['Name']] = filtered_df['County'].iloc[0]

for idx, row in df_3.iterrows():
    if row['Name'] not in city_msa_mapper.keys() and row['County'] != 'NA':
        filtered_df = df_3.loc[df_3['County'] == row['County']]
        
        for idx_1, row_1 in filtered_df.iterrows():
            if row_1['Name'] in city_msa_mapper.keys():
                city_msa_mapper[row['Name']] = city_msa_mapper[row_1['Name']]
    elif row['Name'] not in city_msa_mapper.keys():
        city_msa_mapper[row['Name']] = "NA"


df_final = pd.DataFrame.from_dict(city_msa_mapper, orient='index').reset_index()
df_final.columns = ['City', 'Metro_Politan']
df_final.to_csv("./data/city_metro_politan_mapper.csv" ,index=False)

/var/folders/lr/7v936_cn425434l_8lls3p4m0000gn/T/ipykernel_51810/105523945.py:13: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  filtered_df = df_8.loc[(df_8['State'].str.contains(us_state_to_abbrev[state])) & (df_8['County_name'].str.contains(city_name))]


# Labor Statistics

In [33]:
BLS_ENDPOINT = "https://api.bls.gov/publicAPI/v2/timeseries/data/"
series = ['LNS14000000', 'ENU0608510010']
def fetch_bls_series(series, **kwargs):
    """
    Pass in a list of BLS timeseries to fetch data and return the series
    in JSON format. Arguments can also be provided as kwargs:

        - startyear (4 digit year)
        - endyear (4 digit year)
        - catalog (True or False)
        - calculations (True or False)
        - annualaverage (True or False)
        - registrationKey (api key from BLS website)

    If the registrationKey is not passed in, this function will use the
    BLS_API_KEY fetched from the environment.
    """

    if len(series) < 1 or len(series) > 25:
        raise ValueError("Must pass in between 1 and 25 series ids")

    # Create headers and payload post data
    headers = {'Content-Type': 'application/json'}
    payload = {
        'seriesid': series,
        'registrationKey': BLS_API_KEY,
    }

    # Update the payload with the keyword arguments and convert to JSON
    payload.update(kwargs)
    payload = json.dumps(payload)

    # Fetch the response from the BLS API
    response = requests.post(BLS_ENDPOINT, data=payload, headers=headers)
    response.raise_for_status()

    # Parse the JSON result
    result = response.json()
    if result['status'] != 'REQUEST_SUCCEEDED':
        raise Exception(result['message'][0])

    return result
data = fetch_bls_series(series)
data


# https://catran.sba.gov/apidocs/apidetail.cfm
# headers = {'Content-type': 'application/json'}
# res = requests.get("https://api.bls.gov/publicAPI/v2/timeseries/popular", headers=headers)
# res.json()

# data = json.dumps({"seriesid": ['CUUR0000SA0','SUUR0000SA0'],"startyear":"2011", "endyear":"2014"})
# p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
# json_data = json.loads(p.text)
# for series in json_data['Results']['series']:
#     x=prettytable.PrettyTable(["series id","year","period","value","footnotes"])
#     seriesId = series['seriesID']
#     for item in series['data']:
#         year = item['year']
#         period = item['period']
#         value = item['value']
#         footnotes=""
#         for footnote in item['footnotes']:
#             if footnote:
#                 footnotes = footnotes + footnote['text'] + ','
#         if 'M01' <= period <= 'M12':
#             x.add_row([seriesId,year,period,value,footnotes[0:-1]])
#     output = open(seriesId + '.txt','w')
#     output.write (x.get_string())
#     output.close()

{'status': 'REQUEST_SUCCEEDED',
 'responseTime': 239,
 'message': [],
 'Results': {'series': [{'seriesID': 'LNS14000000',
    'data': [{'year': '2023',
      'period': 'M05',
      'periodName': 'May',
      'latest': 'true',
      'value': '3.7',
      'footnotes': [{}]},
     {'year': '2023',
      'period': 'M04',
      'periodName': 'April',
      'value': '3.4',
      'footnotes': [{}]},
     {'year': '2023',
      'period': 'M03',
      'periodName': 'March',
      'value': '3.5',
      'footnotes': [{}]},
     {'year': '2023',
      'period': 'M02',
      'periodName': 'February',
      'value': '3.6',
      'footnotes': [{}]},
     {'year': '2023',
      'period': 'M01',
      'periodName': 'January',
      'value': '3.4',
      'footnotes': [{}]},
     {'year': '2022',
      'period': 'M12',
      'periodName': 'December',
      'value': '3.5',
      'footnotes': [{}]},
     {'year': '2022',
      'period': 'M11',
      'periodName': 'November',
      'value': '3.6',
      'fo